In [ ]:
# @title Step 1: Install Required Libraries
# This installs the official ElevenLabs Python library and moviepy for video processing.
!pip install -q elevenlabs moviepy

print("✅ Installation complete.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.5 MB/s eta 0:00:00
✅ Installation complete.


In [ ]:
# @title Step 2: Enter API Key & Upload Your Video
from google.colab import files, userdata
import os

# Securely input your ElevenLabs API Key.
# You will be prompted to enter it the first time.
api_key = userdata.get('ELEVENLABS_API_KEY')
if not api_key:
  api_key = input("Please enter your ElevenLabs API Key: ")
  userdata.add('ELEVENLABS_API_KEY', api_key)

# Clean up any previous video files to avoid errors.
if os.path.exists('input_video_with_music.mp4'):
    os.remove('input_video_with_music.mp4')

# Upload the video file you want to clean.
print("\nPlease upload the video file containing the copyrighted music.")
uploaded = files.upload()

# Rename for consistency in the script.
input_filename = next(iter(uploaded))
os.rename(input_filename, 'input_video_with_music.mp4')

print(f"\n✅ Video '{input_filename}' uploaded successfully.")



Please upload the video file containing the copyrighted music.


Saving videoplayback.mp4 to videoplayback.mp4

✅ Video 'videoplayback.mp4' uploaded successfully.


In [ ]:
# @title Step 3: Extract the Original Audio from the Video
from moviepy.editor import VideoFileClip

print("Extracting the original audio (voice + music) from the video...")
video = VideoFileClip("input_video_with_music.mp4")
original_audio_path = "original_audio.mp3"
video.audio.write_audiofile(original_audio_path, verbose=False, logger=None)

print(f"✅ Audio extracted to '{original_audio_path}'.")


Extracting the original audio (voice + music) from the video...
✅ Audio extracted to 'original_audio.mp3'.


In [ ]:
import requests
import os

# Define the API key and audio file path

original_audio_path = "/content/original_audio.mp3"  # Replace with your actual file path, e.g., "C:/Users/YourName/audio.mp3" or "./my_audio.mp3"

print(f"API Key: {api_key}")

# Validate file
if not original_audio_path:
    print(f"❌ Error: 'original_audio_path' is not defined.")
    exit(1)

if not os.path.exists(original_audio_path):
    print(f"❌ Error: Audio file '{original_audio_path}' does not exist.")
    exit(1)

# Verify file format
supported_formats = ['.mp3', '.wav']
file_ext = os.path.splitext(original_audio_path)[1].lower()
if file_ext not in supported_formats:
    print(f"❌ Error: Unsupported file format '{file_ext}'. Use MP3 or WAV.")
    exit(1)

# Define the endpoint explicitly
url = "https://api.elevenlabs.io/v1/audio-isolation"
headers = {"xi-api-key": api_key}

print("Sending audio to ElevenLabs to isolate the voice and remove the music...")

try:
    with open(original_audio_path, "rb") as f:
        # Prepare the file for multipart form data
        files = {
            "audio": (os.path.basename(original_audio_path), f, "audio/mpeg" if file_ext == ".mp3" else "audio/wav")
        }
        response = requests.post(url, headers=headers, files=files)

    if response.status_code == 200:
        isolated_voice_path = "isolated_voice_only.mp3"
        with open(isolated_voice_path, "wb") as f_out:
            f_out.write(response.content)
        print(f"✅ Voice isolated successfully! Saved to '{isolated_voice_path}'.")
    else:
        print(f"❌ API error: {response.status_code} - {response.text}")
        print("Please verify your API key at https://elevenlabs.io/ and ensure you have sufficient credits.")

except Exception as e:
    print(f"❌ An error occurred: {e}")
    print("Please verify your API key and ensure the input audio file is in a supported format (e.g., MP3, WAV) and is not corrupted.")

In [ ]:
# @title Step 5: Create the Final Video with a Clean Audio Track
from moviepy.editor import VideoFileClip, AudioFileClip

if os.path.exists(isolated_voice_path):
    print("Replacing the original audio with the clean, voice-only track...")
    # Load the video (without its audio) and the new clean audio.
    video_clip = VideoFileClip("input_video_with_music.mp4")
    clean_audio_clip = AudioFileClip(isolated_voice_path)

    # Combine them into the final video.
    final_video = video_clip.set_audio(clean_audio_clip)

    # Save the final, copyright-safe video.
    final_video_filename = "final_video_no_music.mp4"
    final_video.write_videofile(final_video_filename, codec="libx264", audio_codec="aac", verbose=False, logger=None)

    print(f"\n✅ Success! Your new video '{final_video_filename}' is ready and safe to upload.")
else:
    print("❌ Cannot create final video because the voice isolation step failed.")



Replacing the original audio with the clean, voice-only track...


  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+




✅ Success! Your new video 'final_video_no_music.mp4' is ready and safe to upload.


In [ ]:
# @title Step 6: Download the Final, Music-Free Video
from google.colab import files

if os.path.exists(final_video_filename):
  print(f"Downloading '{final_video_filename}'...")
  files.download(final_video_filename)
else:
  print("❌ Final video not found. Please ensure all previous steps ran correctly.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# from elevenlabs import ElevenLabs

# client = ElevenLabs(
#     base_url="https://api.elevenlabs.io/"
# )

# client.audio_isolation.convert()

# https://api.elevenlabs.io/v1/audio-isolation